In [1]:
# Seems like we have not stored the predicted values for the test features - the extracted 32 featrues for each subwindow. We need this for the stage two model. Let's not store it here, but we will still need to get these, and let's have a look at this here

In [2]:
import tensorflow as tf
import os
import numpy as np
import pandas as pd

2025-07-23 12:51:07.337747: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-07-23 12:51:07.367842: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
# where is the trained model?

# consider the first non-overlapping model we trained

In [4]:
non_overlapping_model = tf.keras.models.load_model("models/CNN_seq2seq_non_overlapping.keras")

In [5]:
non_overlapping_model.summary()

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 13, None, None, 3)   0         []                            
                             ]                                                                    
                                                                                                  
 time_distributed (TimeDist  (None, 13, 32)               71808     ['input_1[0][0]']             
 ributed)                                                                                         
                                                                                                  
 lstm (LSTM)                 [(None, 64),                 24832     ['time_distributed[0][0]']    
                              (None, 64),                                                   

In [6]:
# Okay, where is our test input features

In [7]:
# input features
input_features_loc = 'data/test_input_sub_images'
input_contents = os.listdir(input_features_loc)
input_contents.sort()

In [8]:
input_contents[0]

'test_data_blk_0103.npy'

In [9]:
# Let's just od this here for a single block, and see how these look like?

In [10]:
block_0103_features = np.load(os.path.join(input_features_loc, input_contents[0]))

In [11]:
block_0103_features.shape

(910, 13, 30, 30, 3)

In [12]:
# get the predictions for this model?

In [13]:
blk_0103_predicted_features = non_overlapping_model.predict(block_0103_features)

29/29 [==============================] - 1s 15ms/step


In [14]:
blk_0103_predicted_features.shape

(910, 7, 32)

In [15]:
# These predicted features seem to be a little high than the features that we have had before for training of the BLAR process model

In [16]:
# let's examine this data for a bit before proceeding - I mean at this point, we can blindly go ahead and run the BLAR model, but let's try and figure out what limitations we have in our data - then we might be able to even fit better models in the first stage to get better final outputs

In [17]:
# Also, notice here, we only have our test data, but for the BLAR model, we also need the 32 features for each sub-image in the sequence - maybe we extract that from the previous model like how it was done ? (May need to check the preprocessing script for comps)

In [18]:
# Let's consider the first sub-image? with index 0 across the 7 test time points
sub_im_0  =blk_0103_predicted_features[0, :, :]

In [19]:
sub_im_0.shape

(7, 32)

In [20]:
# what about acorss time points, I want to know if the 0s occur at the same time points in the 32 features - because this was not the case ealier

In [21]:
t0_sub_im_0 = sub_im_0[0,:]

In [22]:
t0_sub_im_0

array([0.       , 0.       , 0.       , 0.       , 0.       , 1.0394756,
       0.       , 0.9860603, 0.       , 0.       , 0.4001755, 0.       ,
       0.       , 1.2468925, 0.       , 0.       , 0.       , 1.3056968,
       0.       , 1.2609944, 1.1563282, 0.       , 0.       , 0.       ,
       1.1006955, 0.       , 1.1183512, 0.       , 0.       , 1.0843866,
       1.0400674, 0.       ], dtype=float32)

In [23]:
t1_sub_im_0 = sub_im_0[1,:]

In [24]:
t1_sub_im_0

array([0.        , 0.        , 0.        , 0.        , 0.        ,
       1.0432099 , 0.        , 0.9595724 , 0.        , 0.        ,
       0.39496377, 0.        , 0.        , 1.2364255 , 0.        ,
       0.        , 0.        , 1.2660679 , 0.        , 1.2351695 ,
       1.102044  , 0.        , 0.        , 0.        , 1.0570894 ,
       0.        , 1.099581  , 0.        , 0.        , 1.0725211 ,
       1.0299706 , 0.        ], dtype=float32)

In [25]:
# seems like not all the same featrues are 0 for the two time points. - which is good. A little worried about the very high values of exttacted features though, and a lot of zeros as the predicted values (which probably is due to the relu activation function), but let's go ahead and stack the features of all the 7 time points to ensure that not all same features are indeed zeros.

In [26]:
# Get these time-wise features in a for loop? That might be quick and easier to examine
all_time_points_sub_0 = []
for i in range(sub_im_0.shape[0]):
    ti_sub_0 = sub_im_0[i,:]
    all_time_points_sub_0.append(ti_sub_0)

In [27]:
trail_df = pd.DataFrame(all_time_points_sub_0)

In [28]:
# seems like the exact same columns are indeed 0s for all time points, which was not the case earlier
trail_df.head()

,0,1,2,3,4,5,6,7,8,9,...,22,23,24,25,26,27,28,29,30,31
0,0.0,0.0,0.0,0.0,0.0,1.039476,0.0,0.986060,0.0,0.0,...,0.0,0.0,1.100695,0.0,1.118351,0.0,0.0,1.084387,1.040067,0.0
1,0.0,0.0,0.0,0.0,0.0,1.043210,0.0,0.959572,0.0,0.0,...,0.0,0.0,1.057089,0.0,1.099581,0.0,0.0,1.072521,1.029971,0.0
2,0.0,0.0,0.0,0.0,0.0,0.970404,0.0,0.912788,0.0,0.0,...,0.0,0.0,1.004907,0.0,1.034443,0.0,0.0,1.005990,0.951311,0.0
3,0.0,0.0,0.0,0.0,0.0,1.180434,0.0,1.058705,0.0,0.0,...,0.0,0.0,1.201747,0.0,1.234108,0.0,0.0,1.200970,1.166668,0.0
4,0.0,0.0,0.0,0.0,0.0,1.261200,0.0,1.137830,0.0,0.0,...,0.0,0.0,1.283881,0.0,1.333322,0.0,0.0,1.304473,1.250708,0.0


In [29]:
# display all coumns
with pd.option_context('display.max_columns', None):
    display(trail_df)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31
0,0.0,0.0,0.0,0.0,0.0,1.039476,0.0,0.986060,0.0,0.0,0.400176,0.0,0.0,1.246892,0.0,0.0,0.0,1.305697,0.0,1.260994,1.156328,0.0,0.0,0.0,1.100695,0.0,1.118351,0.0,0.0,1.084387,1.040067,0.0
1,0.0,0.0,0.0,0.0,0.0,1.043210,0.0,0.959572,0.0,0.0,0.394964,0.0,0.0,1.236426,0.0,0.0,0.0,1.266068,0.0,1.235170,1.102044,0.0,0.0,0.0,1.057089,0.0,1.099581,0.0,0.0,1.072521,1.029971,0.0
2,0.0,0.0,0.0,0.0,0.0,0.970404,0.0,0.912788,0.0,0.0,0.373518,0.0,0.0,1.166291,0.0,0.0,0.0,1.180260,0.0,1.148413,1.020637,0.0,0.0,0.0,1.004907,0.0,1.034443,0.0,0.0,1.005990,0.951311,0.0
3,0.0,0.0,0.0,0.0,0.0,1.180434,0.0,1.058705,0.0,0.0,0.476584,0.0,0.0,1.377655,0.0,0.0,0.0,1.430579,0.0,1.389209,1.238222,0.0,0.0,0.0,1.201747,0.0,1.234108,0.0,0.0,1.200970,1.166668,0.0
4,0.0,0.0,0.0,0.0,0.0,1.261200,0.0,1.137830,0.0,0.0,0.526926,0.0,0.0,1.492471,0.0,0.0,0.0,1.543945,0.0,1.490151,1.332732,0.0,0.0,0.0,1.283881,0.0,1.333322,0.0,0.0,1.304473,1.250708,0.0
5,0.0,0.0,0.0,0.0,0.0,0.943049,0.0,0.850045,0.0,0.0,0.361572,0.0,0.0,1.123997,0.0,0.0,0.0,1.151053,0.0,1.118078,1.011538,0.0,0.0,0.0,0.945659,0.0,1.005414,0.0,0.0,0.983222,0.947917,0.0
6,0.0,0.0,0.0,0.0,0.0,0.936238,0.0,0.848483,0.0,0.0,0.357935,0.0,0.0,1.120383,0.0,0.0,0.0,1.153662,0.0,1.097126,0.985618,0.0,0.0,0.0,0.963743,0.0,0.996034,0.0,0.0,0.971180,0.924952,0.0


In [30]:
zero_cols = (trail_df == 0).all()
num_zero_cols = zero_cols.sum()
print(f"Number of all-zero columns: {num_zero_cols}")

Number of all-zero columns: 21


In [31]:
# This looks really dreadful - 

In [32]:
# Try another subwindow?
sub_im_100  =blk_0103_predicted_features[100, :, :]

In [33]:
all_time_points_sub_100 = []
for i in range(sub_im_100.shape[0]):
    ti_sub_100 = sub_im_100[i,:]
    all_time_points_sub_100.append(ti_sub_100)

In [34]:
trail_df_100 = pd.DataFrame(all_time_points_sub_100)

In [35]:
trail_df_100

,0,1,2,3,4,5,6,7,8,9,...,22,23,24,25,26,27,28,29,30,31
0,0.0,0.0,0.0,0.0,0.0,0.761100,0.0,0.713808,0.0,0.0,...,0.0,0.0,0.838804,0.0,0.867201,0.0,0.0,0.812566,0.793690,0.0
1,0.0,0.0,0.0,0.0,0.0,0.693636,0.0,0.647874,0.0,0.0,...,0.0,0.0,0.721429,0.0,0.765229,0.0,0.0,0.712197,0.704634,0.0
2,0.0,0.0,0.0,0.0,0.0,0.668664,0.0,0.646977,0.0,0.0,...,0.0,0.0,0.687198,0.0,0.755321,0.0,0.0,0.702020,0.676906,0.0
3,0.0,0.0,0.0,0.0,0.0,1.150124,0.0,1.043255,0.0,0.0,...,0.0,0.0,1.180064,0.0,1.244798,0.0,0.0,1.159636,1.145820,0.0
4,0.0,0.0,0.0,0.0,0.0,1.305720,0.0,1.195615,0.0,0.0,...,0.0,0.0,1.340798,0.0,1.415241,0.0,0.0,1.354784,1.310202,0.0
5,0.0,0.0,0.0,0.0,0.0,1.019596,0.0,0.927067,0.0,0.0,...,0.0,0.0,1.036537,0.0,1.118543,0.0,0.0,1.073477,1.029053,0.0
6,0.0,0.0,0.0,0.0,0.0,0.982463,0.0,0.884909,0.0,0.0,...,0.0,0.0,1.000937,0.0,1.072053,0.0,0.0,1.022905,0.980403,0.0


In [36]:
# display all coumns
with pd.option_context('display.max_columns', None):
    display(trail_df_100)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31
0,0.0,0.0,0.0,0.0,0.0,0.761100,0.0,0.713808,0.0,0.0,0.308646,0.0,0.0,0.963686,0.0,0.0,0.0,0.965786,0.0,0.916343,0.866903,0.0,0.0,0.0,0.838804,0.0,0.867201,0.0,0.0,0.812566,0.793690,0.0
1,0.0,0.0,0.0,0.0,0.0,0.693636,0.0,0.647874,0.0,0.0,0.248946,0.0,0.0,0.829813,0.0,0.0,0.0,0.840235,0.0,0.818859,0.741506,0.0,0.0,0.0,0.721429,0.0,0.765229,0.0,0.0,0.712197,0.704634,0.0
2,0.0,0.0,0.0,0.0,0.0,0.668664,0.0,0.646977,0.0,0.0,0.230199,0.0,0.0,0.823208,0.0,0.0,0.0,0.800127,0.0,0.793992,0.731147,0.0,0.0,0.0,0.687198,0.0,0.755321,0.0,0.0,0.702020,0.676906,0.0
3,0.0,0.0,0.0,0.0,0.0,1.150124,0.0,1.043255,0.0,0.0,0.479051,0.0,0.0,1.336177,0.0,0.0,0.0,1.406905,0.0,1.328961,1.230467,0.0,0.0,0.0,1.180064,0.0,1.244798,0.0,0.0,1.159636,1.145820,0.0
4,0.0,0.0,0.0,0.0,0.0,1.305720,0.0,1.195615,0.0,0.0,0.557260,0.0,0.0,1.554729,0.0,0.0,0.0,1.614214,0.0,1.551459,1.388134,0.0,0.0,0.0,1.340798,0.0,1.415241,0.0,0.0,1.354784,1.310202,0.0
5,0.0,0.0,0.0,0.0,0.0,1.019596,0.0,0.927067,0.0,0.0,0.414082,0.0,0.0,1.230271,0.0,0.0,0.0,1.254359,0.0,1.221898,1.119282,0.0,0.0,0.0,1.036537,0.0,1.118543,0.0,0.0,1.073477,1.029053,0.0
6,0.0,0.0,0.0,0.0,0.0,0.982463,0.0,0.884909,0.0,0.0,0.388291,0.0,0.0,1.182262,0.0,0.0,0.0,1.209084,0.0,1.158109,1.059892,0.0,0.0,0.0,1.000937,0.0,1.072053,0.0,0.0,1.022905,0.980403,0.0


In [37]:
zero_cols = (trail_df_100 == 0).all()
num_zero_cols = zero_cols.sum()
print(f"Number of all-zero columns: {num_zero_cols}")

Number of all-zero columns: 21


In [38]:
# Seems like it's all the same for all subwindows


In [39]:
# Try another subwindow?
sub_im_500  =blk_0103_predicted_features[500, :, :]

In [40]:
all_time_points_sub_500 = []
for i in range(sub_im_500.shape[0]):
    ti_sub_500 = sub_im_500[i,:]
    all_time_points_sub_500.append(ti_sub_500)

In [41]:
trail_df_500 = pd.DataFrame(all_time_points_sub_500)

In [42]:
trail_df_500

,0,1,2,3,4,5,6,7,8,9,...,22,23,24,25,26,27,28,29,30,31
0,0.0,0.0,0.0,0.0,0.0,1.284153,0.0,1.190831,0.0,0.0,...,0.0,0.0,1.332885,0.0,1.354890,0.0,0.0,1.329266,1.273885,0.0
1,0.0,0.0,0.0,0.0,0.0,1.260877,0.0,1.148836,0.0,0.0,...,0.0,0.0,1.282563,0.0,1.320414,0.0,0.0,1.293432,1.232890,0.0
2,0.0,0.0,0.0,0.0,0.0,1.198141,0.0,1.101455,0.0,0.0,...,0.0,0.0,1.225795,0.0,1.267183,0.0,0.0,1.237467,1.167140,0.0
3,0.0,0.0,0.0,0.0,0.0,1.309910,0.0,1.173405,0.0,0.0,...,0.0,0.0,1.322587,0.0,1.371046,0.0,0.0,1.352492,1.289430,0.0
4,0.0,0.0,0.0,0.0,0.0,1.363955,0.0,1.225630,0.0,0.0,...,0.0,0.0,1.388301,0.0,1.435931,0.0,0.0,1.406057,1.335019,0.0
5,0.0,0.0,0.0,0.0,0.0,1.017948,0.0,0.915971,0.0,0.0,...,0.0,0.0,1.028567,0.0,1.070708,0.0,0.0,1.049457,1.001066,0.0
6,0.0,0.0,0.0,0.0,0.0,1.012254,0.0,0.916055,0.0,0.0,...,0.0,0.0,1.036778,0.0,1.074846,0.0,0.0,1.057088,0.988959,0.0


In [43]:
# display all coumns
with pd.option_context('display.max_columns', None):
    display(trail_df_500)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31
0,0.0,0.0,0.0,0.0,0.0,1.284153,0.0,1.190831,0.0,0.0,0.524219,0.0,0.0,1.505396,0.0,0.0,0.0,1.592703,0.0,1.519952,1.389006,0.0,0.0,0.0,1.332885,0.0,1.354890,0.0,0.0,1.329266,1.273885,0.0
1,0.0,0.0,0.0,0.0,0.0,1.260877,0.0,1.148836,0.0,0.0,0.521178,0.0,0.0,1.479625,0.0,0.0,0.0,1.527024,0.0,1.473992,1.328082,0.0,0.0,0.0,1.282563,0.0,1.320414,0.0,0.0,1.293432,1.232890,0.0
2,0.0,0.0,0.0,0.0,0.0,1.198141,0.0,1.101455,0.0,0.0,0.493917,0.0,0.0,1.416483,0.0,0.0,0.0,1.454599,0.0,1.398731,1.251036,0.0,0.0,0.0,1.225795,0.0,1.267183,0.0,0.0,1.237467,1.167140,0.0
3,0.0,0.0,0.0,0.0,0.0,1.309910,0.0,1.173405,0.0,0.0,0.543921,0.0,0.0,1.534417,0.0,0.0,0.0,1.575924,0.0,1.530673,1.369291,0.0,0.0,0.0,1.322587,0.0,1.371046,0.0,0.0,1.352492,1.289430,0.0
4,0.0,0.0,0.0,0.0,0.0,1.363955,0.0,1.225630,0.0,0.0,0.582785,0.0,0.0,1.601119,0.0,0.0,0.0,1.657571,0.0,1.588388,1.428119,0.0,0.0,0.0,1.388301,0.0,1.435931,0.0,0.0,1.406057,1.335019,0.0
5,0.0,0.0,0.0,0.0,0.0,1.017948,0.0,0.915971,0.0,0.0,0.399331,0.0,0.0,1.198558,0.0,0.0,0.0,1.234852,0.0,1.174023,1.070393,0.0,0.0,0.0,1.028567,0.0,1.070708,0.0,0.0,1.049457,1.001066,0.0
6,0.0,0.0,0.0,0.0,0.0,1.012254,0.0,0.916055,0.0,0.0,0.404068,0.0,0.0,1.189654,0.0,0.0,0.0,1.240862,0.0,1.170417,1.067154,0.0,0.0,0.0,1.036778,0.0,1.074846,0.0,0.0,1.057088,0.988959,0.0


In [44]:
zero_cols = (trail_df_500 == 0).all()
num_zero_cols = zero_cols.sum()
print(f"Number of all-zero columns: {num_zero_cols}")

Number of all-zero columns: 21


In [45]:
# So all subwindows have 21 zero columns - this does not seem to be good though